In [1]:
import exasim_plot_helpers as eph
import ipywidgets
from obr.core.queries import query_to_dataframe, query_to_records
from copy import deepcopy
import numpy as np
import pandas as pd

import seaborn
from copy import deepcopy
import matplotlib.pyplot as plt

In [2]:
def post_pro_df(df):
    df["SolvePperIter"] = df["SolveP"]/df["iter_p"]

def generate_df(case):
    folder = "/home/greole/data/code/exasim_project/benchmark_data/" + case
    jobs = eph.signac_conversion.to_jobs(folder)
    q = list(eph.signac_conversion.build_annotated_query()) + list(eph.signac_conversion.build_gko_query("p"))
    df = query_to_dataframe(jobs, q, index=["nCells", "solver", "executor", "nSubDomains", "preconditioner"])
    # df = df.drop(df["TimeStep"].isnull())
    df = df[df['host'].notna()]
    try:
        df.loc[df.campaign == "case", 'campaign'] = "OMPI + HOST_BUFFER"
        df.loc[df.campaign == "host_buffer", 'campaign'] = "MVAPICH + GDR"
    except:
        pass
    return df



dfs = { case:generate_df(case) for case in ["LidDrivenCavity3D", "WindsorBody"]}


cpu_32 = eph.plot_funcs.PlotSpecification(
        legend = "CPU",
        queries =  [
            # eph.helpers.DFQuery(idx="executor", val="CPU"), 
             eph.helpers.DFQuery(idx="preconditioner", val="none"), 
            # eph.helpers.DFQuery(idx="nSubDomains", val=32), 
            ],
        plot_properties = {"linestyle": "-", "color": "red"})

In [13]:
df = deepcopy(dfs["LidDrivenCavity3D"])
df = df.reset_index()

def draw_plot_faceted_plot(x="nCells", y="SolveP", hue="executor", col="host", case="LidDrivenCavity3D", style="nSubDomains", logx=True,logy=True):
    df = deepcopy(dfs[case])
    df = df[eph.helpers.idx_query_mask(df, [eph.helpers.DFQuery(idx="solver", val="GAMG", op=eph.helpers.not_equal())])]
    #df = df[eph.helpers.idx_query_mask(df, [eph.helpers.DFQuery(idx="executor", val="CPU", op=eph.helpers.not_equal())])] 
    df = df.reset_index()
    seaborn.relplot(x=x, y=y, hue=hue, style=style, data=df, col=col, kind="line", markers=True)
    if logx:
        plt.xscale('log')
    if logy:
        plt.yscale('log')
    #g = seaborn.FacetGrid(df, row="solver", col=col, margin_titles=True)
    #g.map(seaborn.relplot, x, y, hue, style,  col=col, kind="line", markers=True)

ipywidgets.interact(
    draw_plot_faceted_plot,
    x=list(df.columns), 
    y=list(df.columns),
    hue=list(df.columns),
    col=list(df.columns),
    case=list(dfs.keys()),
    logy=True,
    logx=True,
)

interactive(children=(Dropdown(description='x', options=('nCells', 'solver', 'executor', 'nSubDomains', 'preco…

<function __main__.draw_plot_faceted_plot(x='nCells', y='SolveP', hue='executor', col='host', case='LidDrivenCavity3D', style='nSubDomains', logx=True, logy=True)>

In [12]:
df = deepcopy(dfs["LidDrivenCavity3D"])
df = df.reset_index()

def draw_plot_faceted_plot(x="nCells", y="SolveP", color="executor", column="host", case="LidDrivenCavity3D", style="nSubDomains", base_idx="executor",  base_val="CPU"):
    df = deepcopy(dfs[case])
    queries = []
    df = df[eph.helpers.idx_query_mask(df, [eph.helpers.DFQuery(idx="preconditioner", val="none", op=eph.helpers.equal())])]
    if base_idx:
        queries = [eph.helpers.DFQuery(idx=base_idx, val=base_val)]
    df = df[eph.helpers.idx_query_mask(df, [eph.helpers.DFQuery(idx="solver", val="GAMG", op=eph.helpers.not_equal())])]
    df = eph.helpers.compute_full_node_normalize(df, cpu_32.queries)
    df.index = df.index.droplevel("host")
    df = df.reset_index()
    ax = seaborn.relplot(x=x, y=y, hue=color, style=style, data=df, col=column, kind="line", markers=True)
    ax.set(ylabel=f"Speepup {y} [-]")
    # plt.xscale('log')


ipywidgets.interact(
    draw_plot_faceted_plot,
    x=list(df.columns), 
    y=list(df.columns),
    color=list(df.columns),
    column=list(df.columns),
    style=list(df.columns),
    case=list(dfs.keys()),
    base_idx="executor",
    base_val="CPU"
)

interactive(children=(Dropdown(description='x', options=('nCells', 'solver', 'executor', 'nSubDomains', 'preco…

<function __main__.draw_plot_faceted_plot(x='nCells', y='SolveP', color='executor', column='host', case='LidDrivenCavity3D', style='nSubDomains', base_idx='executor', base_val='CPU')>

In [5]:
import pandas as pd
def draw_melted_bar_plot(x='nCells', col='executor', sel="nSubDomains", value=""):
    df = deepcopy(dfs["LidDrivenCavity3D"])
    df = df.reset_index()
    melt_idx = ['MomentumPredictor', 'MatrixAssemblyU', 'MatrixAssemblyPI:', 'MatrixAssemblyPII:', 'SolveP']
    df = df[df[sel] == value]
    for m_idx in melt_idx:
        df[m_idx] = df[m_idx]/df['TimeStep']
    df = pd.melt(df,  id_vars=['nCells', 'executor', 'nSubDomains',],  value_vars=melt_idx)
    seaborn.relplot(x=x, y="value", hue="variable", data=df, col=col, kind="line", markers=True)

ipywidgets.interact(
    draw_melted_bar_plot,
    x=["nCells", "nSubDomains"], 
    col=["nCells", "nSubDomains", "executor"],
    sel=["nCells", "nSubDomains", "executor"],
    value=""
)

interactive(children=(Dropdown(description='x', options=('nCells', 'nSubDomains'), value='nCells'), Dropdown(d…

<function __main__.draw_melted_bar_plot(x='nCells', col='executor', sel='nSubDomains', value='')>

In [6]:
import pandas as pd
def draw_melted_bar_plot(x='nCells', col='executor', sel="executor", value="hip"):
    df = deepcopy(dfs["LidDrivenCavity3D"])
    df = df.reset_index()
    melt_idx = [
       'p: update_local_matrix_data:',
       'p: update_non_local_matrix_data:', 'p_matrix: call_update:',
       'p_rhs: call_update:', 'p: init_precond:', 'p: generate_solver:',
       'p: solve:', 'p: copy_x_back:']
    df = df[df[sel] == value]
    for m_idx in melt_idx:
        df[m_idx] = df[m_idx]/df['TimeStep']
    df = pd.melt(df,  id_vars=['nCells', 'executor', 'nSubDomains',],  value_vars=melt_idx)
    seaborn.relplot(x=x, y="value", hue="variable", data=df, col=col, kind="line", markers=True)

ipywidgets.interact(
    draw_melted_bar_plot,
    x=["nCells", "nSubDomains"], 
    col=["nCells", "nSubDomains", "executor"],
    sel=["nCells", "nSubDomains", "executor"],
    value="hip"
)

interactive(children=(Dropdown(description='x', options=('nCells', 'nSubDomains'), value='nCells'), Dropdown(d…

<function __main__.draw_melted_bar_plot(x='nCells', col='executor', sel='executor', value='hip')>

In [8]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
          ['Captive', 'Wild', 'Captive', 'Wild']]
index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.], 'Ref Speed': [2., 2., 4., 4.]},
                  index=index)
df

Max Speed  Ref Speed
Animal Type                         
Falcon Captive      390.0        2.0
       Wild         350.0        2.0
Parrot Captive       30.0        4.0
       Wild          20.0        4.0

In [9]:
def get_ref(x):
    wild_value_query = eph.helpers.DFQuery("Type", "Wild")
    print(eph.helpers.idx_query(x, [wild_value_query]))
    print(x)
    return x
df.groupby(["Ref Speed"]).apply(get_ref)

             Max Speed  Ref Speed
Animal Type                      
Falcon Wild      350.0        2.0
                Max Speed  Ref Speed
Animal Type                         
Falcon Captive      390.0        2.0
       Wild         350.0        2.0
             Max Speed  Ref Speed
Animal Type                      
Parrot Wild       20.0        4.0
                Max Speed  Ref Speed
Animal Type                         
Parrot Captive       30.0        4.0
       Wild          20.0        4.0


/tmp/ipykernel_3069513/2132292774.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df.groupby(["Ref Speed"]).apply(get_ref)


Max Speed  Ref Speed
Animal Type                         
Falcon Captive      390.0        2.0
       Wild         350.0        2.0
Parrot Captive       30.0        4.0
       Wild          20.0        4.0

In [10]:
dfs["LidDrivenCavity3D"]

jobid  \
nCells     solver executor nSubDomains preconditioner                                     
1000000.0  GKOCG  hip      8.0         BJ              aacc754c2b14370e64959b9b20a8fc2e   
                                       none            4e9f3961b69793445a07292ad6db0e4b   
                                       none            0e049029694e0630541fb7febb30db37   
                                       none            1d4690a7a6a9c6668af28ed51df46003   
                           16.0        BJ              55b3c61c3e7cc5109a9cdfa11ab23ab0   
...                                                                                 ...   
64000000.0 PCG    CPU      8.0         none            60b9c3d29d086e917f10fa5f341a6dc1   
                           16.0        none            cc9ce94942d51cfd2233b6f2a3b02172   
                                       none            f702cdcc97987bb0d5ca6472c8d2270e   
                           32.0        none            db8e3e51b62bfa04d36f52ee476e525d   
                                       none            8d477a6bc48e190bb3e21e59f71e3501   

                                                                        host  \
nCells     solver executor nSubDomains preconditioner                          
1000000.0  GKOCG  hip      8.0         BJ              nla-gpu2.ianm.kit.edu   
                                       none            nla-gpu2.ianm.kit.edu   
                                       none            nla-gpu2.ianm.kit.edu   
                                       none            nla-gpu2.ianm.kit.edu   
                           16.0        BJ              nla-gpu2.ianm.kit.edu   
...                                                                      ...   
64000000.0 PCG    CPU      8.0         none            nla-gpu2.ianm.kit.edu   
                           16.0        none            nla-gpu2.ianm.kit.edu   
                                       none            nla-gpu2.ianm.kit.edu   
                           32.0        none            nla-gpu2.ianm.kit.edu   
                                       none            nla-gpu2.ianm.kit.edu   

                                                                 campaign  \
nCells     solver executor nSubDomains preconditioner                       
1000000.0  GKOCG  hip      8.0         BJ                   MVAPICH + GDR   
                                       none                 MVAPICH + GDR   
                                       none            OMPI + HOST_BUFFER   
                                       none            OMPI + HOST_BUFFER   
                           16.0        BJ                   MVAPICH + GDR   
...                                                                   ...   
64000000.0 PCG    CPU      8.0         none            OMPI + HOST_BUFFER   
                           16.0        none                 MVAPICH + GDR   
                                       none            OMPI + HOST_BUFFER   
                           32.0        none            OMPI + HOST_BUFFER   
                                       none                 MVAPICH + GDR   

                                                      tags  \
nCells     solver executor nSubDomains preconditioner        
1000000.0  GKOCG  hip      8.0         BJ              nla   
                                       none            nla   
                                       none                  
                                       none                  
                           16.0        BJ              nla   
...                                                    ...   
64000000.0 PCG    CPU      8.0         none                  
                           16.0        none            nla   
                                       none                  
                           32.0        none                  
                                       none            nla   

                                                

In [7]:
arrays = [
    ["bar", "bar", "bar", "baz", "baz", "qux", "qux", "qux"],
    ["b", "a", "a", "a", "b", "d", "d", "d"],
]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

v = [3,4,2,3,3,3,4,4]
vs = ["a3","a4","a2","b3","b3","c3","c4","c4"]
s = pd.DataFrame({"num":v, "val": vs}, index=index)

In [8]:
s

num val
first second         
bar   b         3  a3
      a         4  a4
      a         2  a2
baz   a         3  b3
      b         3  b3
qux   d         3  c3
      d         4  c4
      d         4  c4

In [9]:
r = s.loc["baz"]
rval = r.select_dtypes(include=np.number)
snonval = s.select_dtypes(exclude=np.number)
sval = s.select_dtypes(include=np.number)

In [10]:
res = sval/rval
res

num
first second          
bar   b       1.000000
      a       1.333333
      a       0.666667
baz   a       1.000000
      b       1.000000
qux   d            NaN
      d            NaN
      d            NaN

In [11]:
res["val"] = snonval["val"]
res

num val
first second              
bar   b       1.000000  a3
      a       1.333333  a4
      a       0.666667  a2
baz   a       1.000000  b3
      b       1.000000  b3
qux   d            NaN  c3
      d            NaN  c4
      d            NaN  c4

In [12]:
snonval.index.is_unique

False

In [24]:
r.index.is_unique

True

In [25]:
rval/sval

num
first second      
bar   b       1.00
      a       0.75
      a       1.50
baz   a       1.00
      b       1.00
qux   d        NaN
      d        NaN
      d        NaN

In [18]:
df

,nCells,solver,executor,nSubDomains,preconditioner,jobid,host,campaign,tags,timestamp,...,cont_error_local,cont_error_cumulative,p: update_local_matrix_data:,p: update_non_local_matrix_data:,p_matrix: call_update:,p_rhs: call_update:,p: init_precond:,p: generate_solver:,p: solve:,p: copy_x_back:
0,1000000.0,GKOCG,hip,8.0,BJ,aacc754c2b14370e64959b9b20a8fc2e,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_21:34:22,...,1.401410e-08,1.416940e-20,1.611053,0.039947,2.385368,0.511421,0.177263,0.002105,176.721895,0.374526
1,1000000.0,GKOCG,hip,8.0,none,4e9f3961b69793445a07292ad6db0e4b,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_16:04:03,...,1.289780e-08,1.594870e-20,1.669895,0.042474,1.984263,1.282947,0.044211,0.004316,175.584684,0.377579
2,1000000.0,GKOCG,hip,8.0,none,0e049029694e0630541fb7febb30db37,nla-gpu2.ianm.kit.edu,case,,2023-06-26_15:12:26,...,1.250990e-08,-2.608710e-20,5.283053,0.081421,2.775000,1.328474,0.033789,0.002105,307.271105,0.455737
3,1000000.0,GKOCG,hip,8.0,none,1d4690a7a6a9c6668af28ed51df46003,nla-gpu2.ianm.kit.edu,case,,2023-06-27_04:32:40,...,1.432220e-08,-9.886160e-20,5.361158,0.078474,2.846263,0.826105,0.035632,0.001789,309.884211,0.454105
4,1000000.0,GKOCG,hip,16.0,BJ,55b3c61c3e7cc5109a9cdfa11ab23ab0,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_21:35:58,...,1.405580e-08,-5.306950e-20,0.911000,0.023421,1.884211,0.423579,0.163895,0.003105,197.718053,0.197053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,64000000.0,PCG,CPU,8.0,none,60b9c3d29d086e917f10fa5f341a6dc1,nla-gpu2.ianm.kit.edu,case,,2023-06-26_16:49:17,...,1.350360e-07,1.337470e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,64000000.0,PCG,CPU,16.0,none,cc9ce94942d51cfd2233b6f2a3b02172,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-03-26_14:34:01,...,1.328890e-07,3.123760e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,64000000.0,PCG,CPU,16.0,none,f702cdcc97987bb0d5ca6472c8d2270e,nla-gpu2.ianm.kit.edu,case,,2023-06-27_04:33:03,...,1.328890e-07,3.123760e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,64000000.0,PCG,CPU,32.0,none,db8e3e51b62bfa04d36f52ee476e525d,nla-gpu2.ianm.kit.edu,case,,2023-06-26_12:11:17,...,1.336480e-07,2.760780e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df[df['host'].notna()]

,nCells,solver,executor,nSubDomains,preconditioner,jobid,host,campaign,tags,timestamp,...,cont_error_local,cont_error_cumulative,p: update_local_matrix_data:,p: update_non_local_matrix_data:,p_matrix: call_update:,p_rhs: call_update:,p: init_precond:,p: generate_solver:,p: solve:,p: copy_x_back:
0,1000000.0,GKOCG,hip,8.0,BJ,aacc754c2b14370e64959b9b20a8fc2e,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_21:34:22,...,1.401410e-08,1.416940e-20,1.611053,0.039947,2.385368,0.511421,0.177263,0.002105,176.721895,0.374526
1,1000000.0,GKOCG,hip,8.0,none,4e9f3961b69793445a07292ad6db0e4b,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_16:04:03,...,1.289780e-08,1.594870e-20,1.669895,0.042474,1.984263,1.282947,0.044211,0.004316,175.584684,0.377579
2,1000000.0,GKOCG,hip,8.0,none,0e049029694e0630541fb7febb30db37,nla-gpu2.ianm.kit.edu,case,,2023-06-26_15:12:26,...,1.250990e-08,-2.608710e-20,5.283053,0.081421,2.775000,1.328474,0.033789,0.002105,307.271105,0.455737
3,1000000.0,GKOCG,hip,8.0,none,1d4690a7a6a9c6668af28ed51df46003,nla-gpu2.ianm.kit.edu,case,,2023-06-27_04:32:40,...,1.432220e-08,-9.886160e-20,5.361158,0.078474,2.846263,0.826105,0.035632,0.001789,309.884211,0.454105
4,1000000.0,GKOCG,hip,16.0,BJ,55b3c61c3e7cc5109a9cdfa11ab23ab0,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-07-05_21:35:58,...,1.405580e-08,-5.306950e-20,0.911000,0.023421,1.884211,0.423579,0.163895,0.003105,197.718053,0.197053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,64000000.0,PCG,CPU,8.0,none,60b9c3d29d086e917f10fa5f341a6dc1,nla-gpu2.ianm.kit.edu,case,,2023-06-26_16:49:17,...,1.350360e-07,1.337470e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,64000000.0,PCG,CPU,16.0,none,cc9ce94942d51cfd2233b6f2a3b02172,nla-gpu2.ianm.kit.edu,host_buffer,nla,2023-03-26_14:34:01,...,1.328890e-07,3.123760e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,64000000.0,PCG,CPU,16.0,none,f702cdcc97987bb0d5ca6472c8d2270e,nla-gpu2.ianm.kit.edu,case,,2023-06-27_04:33:03,...,1.328890e-07,3.123760e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,64000000.0,PCG,CPU,32.0,none,db8e3e51b62bfa04d36f52ee476e525d,nla-gpu2.ianm.kit.edu,case,,2023-06-26_12:11:17,...,1.336480e-07,2.760780e-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
